# Notebook for the project in *TDT4259 - Applied Data Science*

Necessary imports

In [1]:
import sys
import os

import pandas as pd
import numpy as np
sys.path.append(".")

from util.util import get_dataset, clean_attrition_dataset, create_additional_columns

We start by preprocessing the columns to be on a concise format.

In [2]:
raw_dataset = get_dataset("train_data.csv")
cleaned_dataset = clean_attrition_dataset(raw_dataset)
final_dataset = create_additional_columns(cleaned_dataset)